In [6]:
path='C:/Users/student/Downloads/데이터들/'

In [2]:
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, Dropout
from keras.optimizers import *
from keras.utils import np_utils
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer

from nltk.tokenize import sent_tokenize
from konlpy.corpus import kolaw
from konlpy.tag import Okt

Using TensorFlow backend.


In [3]:
chatbotData=pd.read_csv('C:/Users/student/Downloads/Python_JP/ChatData.csv')
question, answer = list(chatbotData['Q']), list(chatbotData['A'])

In [4]:
len(answer)

11823

In [5]:
c=kolaw.open('constitution.txt').read()

In [6]:
sent_tokenize(c) # 문장단위로 

['대한민국헌법\n\n유구한 역사와 전통에 빛나는 우리 대한국민은 3·1운동으로 건립된 대한민국임시정부의 법통과 불의에 항거한 4·19민주이념을 계승하고, 조국의 민주개혁과 평화적 통일의 사명에 입각하여 정의·인도와 동포애로써 민족의 단결을 공고히 하고, 모든 사회적 폐습과 불의를 타파하며, 자율과 조화를 바탕으로 자유민주적 기본질서를 더욱 확고히 하여 정치·경제·사회·문화의 모든 영역에 있어서 각인의 기회를 균등히 하고, 능력을 최고도로 발휘하게 하며, 자유와 권리에 따르는 책임과 의무를 완수하게 하여, 안으로는 국민생활의 균등한 향상을 기하고 밖으로는 항구적인 세계평화와 인류공영에 이바지함으로써 우리들과 우리들의 자손의 안전과 자유와 행복을 영원히 확보할 것을 다짐하면서 1948년 7월 12일에 제정되고 8차에 걸쳐 개정된 헌법을 이제 국회의 의결을 거쳐 국민투표에 의하여 개정한다.',
 '제1장 총강\n  제1조 ① 대한민국은 민주공화국이다.',
 '②대한민국의 주권은 국민에게 있고, 모든 권력은 국민으로부터 나온다.',
 '제2조 ① 대한민국의 국민이 되는 요건은 법률로 정한다.',
 '②국가는 법률이 정하는 바에 의하여 재외국민을 보호할 의무를 진다.',
 '제3조 대한민국의 영토는 한반도와 그 부속도서로 한다.',
 '제4조 대한민국은 통일을 지향하며, 자유민주적 기본질서에 입각한 평화적 통일 정책을 수립하고 이를 추진한다.',
 '제5조 ① 대한민국은 국제평화의 유지에 노력하고 침략적 전쟁을 부인한다.',
 '②국군은 국가의 안전보장과 국토방위의 신성한 의무를 수행함을 사명으로 하며, 그 정치적 중립성은 준수된다.',
 '제6조 ① 헌법에 의하여 체결·공포된 조약과 일반적으로 승인된 국제법규는 국내법과 같은 효력을 가진다.',
 '②외국인은 국제법과 조약이 정하는 바에 의하여 그 지위가 보장된다.',
 '제7조 ① 공무원은 국민전체에 대한 봉사자이며, 국민에 대하여 책임을 진다.',
 '②공무원의 신분과 정치적 중립성은 법률이 정하는 바에 의하여 

In [7]:
okt=Okt()
okt.pos("오늘은 날씨가 따뜻합니다")

[('오늘', 'Noun'),
 ('은', 'Josa'),
 ('날씨', 'Noun'),
 ('가', 'Josa'),
 ('따뜻합니다', 'Adjective')]

In [8]:
doc0=[" ".join([''.join(w) for w, t in okt.pos(s) 
                if t not in ['Number','Foreign'] and w not in ['제','조']])for s in sent_tokenize(c)]

In [9]:
tokenizer=Tokenizer()
tokenizer.word_index

{}

In [10]:
tokenizer.fit_on_texts(doc0)
doc=[tok for tok in tokenizer.texts_to_sequences(doc0) if len(tok)>1]

In [11]:
maxLen=max([len(x)-1 for x in doc])
print(maxLen)
vocabSize=len(tokenizer.word_index)+1
print(vocabSize)

187
1165


In [12]:
sent=[s for s in sent_tokenize(c)]
sent[0]

'대한민국헌법\n\n유구한 역사와 전통에 빛나는 우리 대한국민은 3·1운동으로 건립된 대한민국임시정부의 법통과 불의에 항거한 4·19민주이념을 계승하고, 조국의 민주개혁과 평화적 통일의 사명에 입각하여 정의·인도와 동포애로써 민족의 단결을 공고히 하고, 모든 사회적 폐습과 불의를 타파하며, 자율과 조화를 바탕으로 자유민주적 기본질서를 더욱 확고히 하여 정치·경제·사회·문화의 모든 영역에 있어서 각인의 기회를 균등히 하고, 능력을 최고도로 발휘하게 하며, 자유와 권리에 따르는 책임과 의무를 완수하게 하여, 안으로는 국민생활의 균등한 향상을 기하고 밖으로는 항구적인 세계평화와 인류공영에 이바지함으로써 우리들과 우리들의 자손의 안전과 자유와 행복을 영원히 확보할 것을 다짐하면서 1948년 7월 12일에 제정되고 8차에 걸쳐 개정된 헌법을 이제 국회의 의결을 거쳐 국민투표에 의하여 개정한다.'

In [13]:
sent_tokenize(c)

['대한민국헌법\n\n유구한 역사와 전통에 빛나는 우리 대한국민은 3·1운동으로 건립된 대한민국임시정부의 법통과 불의에 항거한 4·19민주이념을 계승하고, 조국의 민주개혁과 평화적 통일의 사명에 입각하여 정의·인도와 동포애로써 민족의 단결을 공고히 하고, 모든 사회적 폐습과 불의를 타파하며, 자율과 조화를 바탕으로 자유민주적 기본질서를 더욱 확고히 하여 정치·경제·사회·문화의 모든 영역에 있어서 각인의 기회를 균등히 하고, 능력을 최고도로 발휘하게 하며, 자유와 권리에 따르는 책임과 의무를 완수하게 하여, 안으로는 국민생활의 균등한 향상을 기하고 밖으로는 항구적인 세계평화와 인류공영에 이바지함으로써 우리들과 우리들의 자손의 안전과 자유와 행복을 영원히 확보할 것을 다짐하면서 1948년 7월 12일에 제정되고 8차에 걸쳐 개정된 헌법을 이제 국회의 의결을 거쳐 국민투표에 의하여 개정한다.',
 '제1장 총강\n  제1조 ① 대한민국은 민주공화국이다.',
 '②대한민국의 주권은 국민에게 있고, 모든 권력은 국민으로부터 나온다.',
 '제2조 ① 대한민국의 국민이 되는 요건은 법률로 정한다.',
 '②국가는 법률이 정하는 바에 의하여 재외국민을 보호할 의무를 진다.',
 '제3조 대한민국의 영토는 한반도와 그 부속도서로 한다.',
 '제4조 대한민국은 통일을 지향하며, 자유민주적 기본질서에 입각한 평화적 통일 정책을 수립하고 이를 추진한다.',
 '제5조 ① 대한민국은 국제평화의 유지에 노력하고 침략적 전쟁을 부인한다.',
 '②국군은 국가의 안전보장과 국토방위의 신성한 의무를 수행함을 사명으로 하며, 그 정치적 중립성은 준수된다.',
 '제6조 ① 헌법에 의하여 체결·공포된 조약과 일반적으로 승인된 국제법규는 국내법과 같은 효력을 가진다.',
 '②외국인은 국제법과 조약이 정하는 바에 의하여 그 지위가 보장된다.',
 '제7조 ① 공무원은 국민전체에 대한 봉사자이며, 국민에 대하여 책임을 진다.',
 '②공무원의 신분과 정치적 중립성은 법률이 정하는 바에 의하여 

In [14]:
# konlpy.rog => API => Okt
temp=sent_tokenize(c)[0:5]
for i in range(10):
    for w,t in okt.pos(temp[i]): # 형태소로 분리하는 함수
        print('w:',w)
        print("t:",t)

w: 대한민국
t: Noun
w: 헌법
t: Noun
w: 


t: Foreign
w: 유구
t: Noun
w: 한
t: Josa
w: 역사
t: Noun
w: 와
t: Josa
w: 전통
t: Noun
w: 에
t: Josa
w: 빛나는
t: Verb
w: 우리
t: Noun
w: 대
t: Modifier
w: 한
t: Modifier
w: 국민
t: Noun
w: 은
t: Josa
w: 3
t: Number
w: ·
t: Punctuation
w: 1
t: Number
w: 운동
t: Noun
w: 으로
t: Josa
w: 건립
t: Noun
w: 된
t: Verb
w: 대한민국
t: Noun
w: 임시정부
t: Noun
w: 의
t: Josa
w: 법
t: Noun
w: 통과
t: Noun
w: 불의
t: Noun
w: 에
t: Josa
w: 항거
t: Noun
w: 한
t: Josa
w: 4
t: Number
w: ·
t: Punctuation
w: 19
t: Number
w: 민주
t: Noun
w: 이념
t: Noun
w: 을
t: Josa
w: 계승
t: Noun
w: 하고
t: Josa
w: ,
t: Punctuation
w: 조국
t: Noun
w: 의
t: Josa
w: 민주
t: Noun
w: 개혁
t: Noun
w: 과
t: Josa
w: 평화
t: Noun
w: 적
t: Suffix
w: 통일
t: Noun
w: 의
t: Josa
w: 사명
t: Noun
w: 에
t: Josa
w: 입
t: Noun
w: 각하
t: Noun
w: 여
t: Josa
w: 정의
t: Noun
w: ·
t: Punctuation
w: 인도
t: Noun
w: 와
t: Josa
w: 동포
t: Noun
w: 애
t: Noun
w: 로써
t: Noun
w: 민족
t: Noun
w: 의
t: Josa
w: 단결
t: Noun
w: 을
t: Josa
w: 공고
t: Noun
w: 히
t: Adverb
w: 하고
t: Verb
w: ,
t: Punctuation
w

IndexError: list index out of range

In [15]:
tokenizer=Tokenizer()
tokenizer.fit_on_texts(doc0)

In [16]:
len(tokenizer.index_word)
tokenizer.word_index

{'의': 1,
 '에': 2,
 '을': 3,
 '은': 4,
 '이': 5,
 '한다': 6,
 '·': 7,
 '를': 8,
 '법률': 9,
 '는': 10,
 '한': 11,
 '할': 12,
 '정': 13,
 '수': 14,
 '대통령': 15,
 '과': 16,
 '국가': 17,
 '또는': 18,
 '국회': 19,
 '의하여': 20,
 '하는': 21,
 '국민': 22,
 '하여': 23,
 '로': 24,
 '관': 25,
 '있다': 26,
 '때': 27,
 '헌법': 28,
 '그': 29,
 '에는': 30,
 '가': 31,
 '으로': 32,
 '와': 33,
 '모든': 34,
 '바': 35,
 '위': 36,
 '적': 37,
 '하며': 38,
 '하여야': 39,
 '된': 40,
 '아니': 41,
 '기타': 42,
 '가진다': 43,
 '및': 44,
 '장': 45,
 '사항': 46,
 '인': 47,
 '에서': 48,
 '권리': 49,
 '안': 50,
 '된다': 51,
 '필요한': 52,
 '없다': 53,
 '의원': 54,
 '자유': 55,
 '정부': 56,
 '선거': 57,
 '임명': 58,
 '하고': 59,
 '되지': 60,
 '직무': 61,
 '국무총리': 62,
 '경제': 63,
 '의무': 64,
 '조직': 65,
 '자': 66,
 '하': 67,
 '국회의원': 68,
 '임기': 69,
 '의결': 70,
 '공무원': 71,
 '경우': 72,
 '이상': 73,
 '국무위원': 74,
 '대법원': 75,
 '하기': 76,
 '범위': 77,
 '이를': 78,
 '다': 79,
 '될': 80,
 '받지': 81,
 '법관': 82,
 '보호': 83,
 '정책': 84,
 '정당': 85,
 '헌법재판소': 86,
 '재적': 87,
 '회의': 88,
 '항의': 89,
 '관리': 90,
 '진다': 91,
 '노력': 92,
 '보장': 93,
 

In [17]:
tokenizer.texts_to_sequences(doc0)

[[102,
  28,
  602,
  11,
  603,
  33,
  604,
  2,
  605,
  323,
  606,
  11,
  22,
  4,
  7,
  427,
  32,
  607,
  40,
  102,
  608,
  1,
  324,
  609,
  428,
  2,
  610,
  11,
  7,
  177,
  611,
  3,
  429,
  59,
  325,
  1,
  177,
  612,
  16,
  178,
  37,
  179,
  1,
  430,
  2,
  613,
  614,
  110,
  615,
  7,
  616,
  33,
  617,
  618,
  215,
  326,
  1,
  619,
  3,
  327,
  620,
  59,
  34,
  136,
  37,
  621,
  16,
  428,
  8,
  622,
  38,
  431,
  16,
  432,
  8,
  623,
  32,
  433,
  37,
  137,
  259,
  138,
  8,
  624,
  625,
  23,
  122,
  7,
  63,
  7,
  136,
  7,
  260,
  1,
  34,
  261,
  2,
  123,
  626,
  1,
  328,
  8,
  627,
  59,
  329,
  3,
  434,
  628,
  629,
  262,
  38,
  55,
  33,
  49,
  2,
  630,
  216,
  16,
  64,
  8,
  631,
  262,
  23,
  50,
  435,
  22,
  139,
  1,
  436,
  217,
  3,
  632,
  263,
  633,
  435,
  634,
  37,
  47,
  635,
  178,
  33,
  636,
  637,
  2,
  638,
  218,
  264,
  323,
  437,
  16,
  323,
  437,
  1,
  639,
  1,
  640,
  16,
 

In [18]:
np.shape(tokenizer.texts_to_sequences(doc0))

(357,)

In [19]:
# for a in tokenizer.texts_to_sequences(doc0):
#     if len(a)<=1:
#         print(len(a))
doc=[tok for tok in tokenizer.texts_to_sequences(doc0) if len(tok)>1]     


In [20]:
#maxLen=max([len(x) for x in doc])
#tokenizer.word_counts    
vocabSize=len(tokenizer.word_index)+1
print(maxLen, vocabSize) # 187, 1165

187 1165


In [21]:
doc

[[102,
  28,
  602,
  11,
  603,
  33,
  604,
  2,
  605,
  323,
  606,
  11,
  22,
  4,
  7,
  427,
  32,
  607,
  40,
  102,
  608,
  1,
  324,
  609,
  428,
  2,
  610,
  11,
  7,
  177,
  611,
  3,
  429,
  59,
  325,
  1,
  177,
  612,
  16,
  178,
  37,
  179,
  1,
  430,
  2,
  613,
  614,
  110,
  615,
  7,
  616,
  33,
  617,
  618,
  215,
  326,
  1,
  619,
  3,
  327,
  620,
  59,
  34,
  136,
  37,
  621,
  16,
  428,
  8,
  622,
  38,
  431,
  16,
  432,
  8,
  623,
  32,
  433,
  37,
  137,
  259,
  138,
  8,
  624,
  625,
  23,
  122,
  7,
  63,
  7,
  136,
  7,
  260,
  1,
  34,
  261,
  2,
  123,
  626,
  1,
  328,
  8,
  627,
  59,
  329,
  3,
  434,
  628,
  629,
  262,
  38,
  55,
  33,
  49,
  2,
  630,
  216,
  16,
  64,
  8,
  631,
  262,
  23,
  50,
  435,
  22,
  139,
  1,
  436,
  217,
  3,
  632,
  263,
  633,
  435,
  634,
  37,
  47,
  635,
  178,
  33,
  636,
  637,
  2,
  638,
  218,
  264,
  323,
  437,
  16,
  323,
  437,
  1,
  639,
  1,
  640,
  16,
 

In [22]:
def genData(doc, maxLen, vocabSize):
    for sent in doc:
        inputs=[]
        targets=[]
        for i in range(1, len(sent)): # 1~6
            inputs.append(sent[0:i]) # [[45], [45,139] ...
            targets.append(sent[i])  # [439], [648] ...
        y=np_utils.to_categorical(targets, vocabSize)
        inputSeq=sequence.pad_sequences(inputs, maxlen=maxLen)
        yield(inputSeq,y) # 객체가 리턴 됨
#         print("inputs:",inputs)
#         print("-"*50)
#         print("targets:",targets)
#         print("="*50)

In [23]:
for i, (x,y) in enumerate(genData(doc, maxLen, vocabSize)):
    print(i)
    print("x",x.shape, "\n",x)
    print("y",y.shape, '\n',y)

0
x (187, 187) 
 [[  0   0   0 ...   0   0 102]
 [  0   0   0 ...   0 102  28]
 [  0   0   0 ... 102  28 602]
 ...
 [  0   0 102 ... 647 155   2]
 [  0 102  28 ... 155   2  20]
 [102  28 602 ...   2  20 180]]
y (187, 1165) 
 [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
1
x (6, 187) 
 [[  0   0   0 ...   0   0  45]
 [  0   0   0 ...   0  45 439]
 [  0   0   0 ...  45 439 648]
 [  0   0   0 ... 439 648 102]
 [  0   0   0 ... 648 102   4]
 [  0   0   0 ... 102   4 649]]
y (6, 1165) 
 [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
2
x (12, 187) 
 [[  0   0   0 ...   0   0 102]
 [  0   0   0 ...   0 102   1]
 [  0   0   0 ... 102   1 440]
 ...
 [  0   0   0 ...  34 651   4]
 [  0   0   0 ... 651   4  22]
 [  0   0   0 ...   4  22 331]]
y (12, 1165) 
 [[0. 1. 0. ... 0. 0. 0.]
 [0.

y (17, 1165) 
 [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
83
x (14, 187) 
 [[  0   0   0 ...   0   0 131]
 [  0   0   0 ...   0 131 297]
 [  0   0   0 ... 131 297   1]
 ...
 [  0   0   0 ...  93 796   9]
 [  0   0   0 ... 796   9  24]
 [  0   0   0 ...   9  24  13]]
y (14, 1165) 
 [[0. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
84
x (21, 187) 
 [[  0   0   0 ...   0   0 501]
 [  0   0   0 ...   0 501   1]
 [  0   0   0 ... 501   1 131]
 ...
 [  0   0   0 ... 469 453   3]
 [  0   0   0 ... 453   3  81]
 [  0   0   0 ...   3  81  41]]
y (21, 1165) 
 [[0. 1. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
85
x (7, 187) 
 [[  0   0   0 ...   0   0 797]
 [  0   0   0 ...   0 

 [  0   0   0 ...  70  12  14]]
y (55, 1165) 
 [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
165
x (25, 187) 
 [[  0   0   0 ...   0   0  89]
 [  0   0   0 ...   0  89 108]
 [  0   0   0 ...  89 108 163]
 ...
 [  0   0   0 ...  99   1 101]
 [  0   0   0 ...   1 101   5]
 [  0   0   0 ... 101   5 204]]
y (25, 1165) 
 [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
166
x (23, 187) 
 [[  0   0   0 ...   0   0 104]
 [  0   0   0 ...   0 104  15]
 [  0   0   0 ... 104  15   2]
 ...
 [  0   0   0 ...  73   1 101]
 [  0   0   0 ...   1 101   5]
 [  0   0   0 ... 101   5 204]]
y (23, 1165) 
 [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
167
x (19, 187) 
 [[  0   0   0 ...

y (11, 1165) 
 [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
241
x (10, 187) 
 [[  0   0   0 ...   0   0 104]
 [  0   0   0 ...   0 104 570]
 [  0   0   0 ... 104 570  15]
 ...
 [  0   0   0 ...  27  30  15]
 [  0   0   0 ...  30  15   5]
 [  0   0   0 ...  15   5 413]]
y (10, 1165) 
 [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
242
x (17, 187) 
 [[  0   0   0 ...   0   0  17]
 [  0   0   0 ...   0  17 318]
 [  0   0   0 ...  17 318  24]
 ...
 [  0   0   0 ...  46   4   9]
 [  0   0   0 ...   4   9  24]
 [  0   0   0 ...   9  24  13]]
y (17, 1165) 
 [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
243
x (35, 187) 
 [[  0   0   0 ...   0   0  17]
 [  0   0   0 ... 

     0  175    2 1078    8]]
y (4, 1165) 
 [[0. 0. 1. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
317
x (28, 187) 
 [[   0    0    0 ...    0    0 1079]
 [   0    0    0 ...    0 1079    1]
 [   0    0    0 ... 1079    1   65]
 ...
 [   0    0    0 ...   46    4    9]
 [   0    0    0 ...    4    9   24]
 [   0    0    0 ...    9   24   13]]
y (28, 1165) 
 [[0. 1. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
318
x (22, 187) 
 [[   0    0    0 ...    0    0   45]
 [   0    0    0 ...    0   45   63]
 [   0    0    0 ...   45   63  102]
 ...
 [   0    0    0 ... 1082  218    3]
 [   0    0    0 ...  218    3  222]
 [   0    0    0 ...    3  222   32]]
y (22, 1165) 
 [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
319
x (51, 187) 
 [[  

In [24]:
def cf():
    for i in range(3):
        yield i*i
        
print(cf())

<generator object cf at 0x000002B6F560D648>


In [25]:
genData(doc, maxLen, vocabSize)

<generator object genData at 0x000002B6F560D348>

In [26]:
xdata=[]
ydata=[]
for x,y in genData(doc, maxLen, vocabSize):
    xdata.append(x)
    ydata.append(y)
xdata=np.concatenate(xdata)
ydata=np.concatenate(ydata)

In [27]:
# xdata
# 배열 합치기
a=np.array([[1,2],[3,4]])
b=np.array([[5,6],[7,8],[9,10]])
np.concatenate((a,b), axis=0)
np.concatenate((a,b), axis=None)

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10])

In [28]:
ydata.shape

(6917, 1165)

In [29]:
model=Sequential()
model.add(Embedding(vocabSize, 100, input_length=maxLen)) # 1165,100
model.add(LSTM(100, return_sequences=False))
model.add(Dropout(0,5))
model.add(Dense(vocabSize, activation='softmax'))

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [30]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 187, 100)          116500    
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dropout_1 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 1165)              117665    
Total params: 314,565
Trainable params: 314,565
Non-trainable params: 0
_________________________________________________________________


In [31]:
model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

In [32]:
hist=model.fit(xdata, ydata, epochs=500, batch_size=800)


Epoch 1/500
6917/6917 [==============================] - 16s 2ms/step - loss: 6.6592 - accuracy: 0.0288
Epoch 2/500
6917/6917 [==============================] - 17s 2ms/step - loss: 5.8072 - accuracy: 0.0538
Epoch 3/500
6917/6917 [==============================] - 16s 2ms/step - loss: 5.6895 - accuracy: 0.0538
Epoch 4/500
6917/6917 [==============================] - 17s 2ms/step - loss: 5.6724 - accuracy: 0.0538
Epoch 5/500
6917/6917 [==============================] - 16s 2ms/step - loss: 5.6659 - accuracy: 0.0538
Epoch 6/500
6917/6917 [==============================] - 16s 2ms/step - loss: 5.6555 - accuracy: 0.0538
Epoch 7/500
6917/6917 [==============================] - 17s 2ms/step - loss: 5.6368 - accuracy: 0.0538
Epoch 8/500
6917/6917 [==============================] - 17s 2ms/step - loss: 5.6130 - accuracy: 0.0538
Epoch 9/500
6917/6917 [==============================] - 17s 2ms/step - loss: 5.5783 - accuracy: 0.0538
Epoch 10/500
6917/6917 [==============================] - 17s 2

6917/6917 [==============================] - 17s 3ms/step - loss: 3.2084 - accuracy: 0.3416
Epoch 78/500
6917/6917 [==============================] - 17s 3ms/step - loss: 3.1767 - accuracy: 0.3468
Epoch 79/500
6917/6917 [==============================] - 17s 3ms/step - loss: 3.1497 - accuracy: 0.3529
Epoch 80/500
6917/6917 [==============================] - 17s 3ms/step - loss: 3.1230 - accuracy: 0.3561
Epoch 81/500
6917/6917 [==============================] - 17s 3ms/step - loss: 3.0914 - accuracy: 0.3598
Epoch 82/500
6917/6917 [==============================] - 18s 3ms/step - loss: 3.0663 - accuracy: 0.3662
Epoch 83/500
6917/6917 [==============================] - 17s 3ms/step - loss: 3.0362 - accuracy: 0.3700
Epoch 84/500
6917/6917 [==============================] - 17s 3ms/step - loss: 3.0076 - accuracy: 0.3762
Epoch 85/500
6917/6917 [==============================] - 17s 3ms/step - loss: 2.9832 - accuracy: 0.3809
Epoch 86/500
6917/6917 [==============================] - 17s 3ms/st

6917/6917 [==============================] - 22s 3ms/step - loss: 1.5468 - accuracy: 0.6658
Epoch 155/500
6917/6917 [==============================] - 22s 3ms/step - loss: 1.5314 - accuracy: 0.6694
Epoch 156/500
6917/6917 [==============================] - 22s 3ms/step - loss: 1.5193 - accuracy: 0.6727
Epoch 157/500
6917/6917 [==============================] - 22s 3ms/step - loss: 1.5037 - accuracy: 0.6763
Epoch 158/500
6917/6917 [==============================] - 22s 3ms/step - loss: 1.4924 - accuracy: 0.6818
Epoch 159/500
6917/6917 [==============================] - 22s 3ms/step - loss: 1.4763 - accuracy: 0.6830
Epoch 160/500
6917/6917 [==============================] - 23s 3ms/step - loss: 1.4602 - accuracy: 0.6889
Epoch 161/500
6917/6917 [==============================] - 23s 3ms/step - loss: 1.4465 - accuracy: 0.6922
Epoch 162/500
6917/6917 [==============================] - 22s 3ms/step - loss: 1.4344 - accuracy: 0.6922
Epoch 163/500
6917/6917 [==============================] - 2

6917/6917 [==============================] - 22s 3ms/step - loss: 0.7416 - accuracy: 0.8530
Epoch 232/500
6917/6917 [==============================] - 22s 3ms/step - loss: 0.7363 - accuracy: 0.8524
Epoch 233/500
6917/6917 [==============================] - 22s 3ms/step - loss: 0.7284 - accuracy: 0.8547
Epoch 234/500
6917/6917 [==============================] - 22s 3ms/step - loss: 0.7234 - accuracy: 0.8569
Epoch 235/500
6917/6917 [==============================] - 22s 3ms/step - loss: 0.7168 - accuracy: 0.8589
Epoch 236/500
6917/6917 [==============================] - 22s 3ms/step - loss: 0.7081 - accuracy: 0.8619
Epoch 237/500
6917/6917 [==============================] - 22s 3ms/step - loss: 0.6994 - accuracy: 0.8616
Epoch 238/500
6917/6917 [==============================] - 22s 3ms/step - loss: 0.6976 - accuracy: 0.8605
Epoch 239/500
6917/6917 [==============================] - 22s 3ms/step - loss: 0.6900 - accuracy: 0.8638
Epoch 240/500
6917/6917 [==============================] - 2

6917/6917 [==============================] - 22s 3ms/step - loss: 0.3668 - accuracy: 0.9293
Epoch 309/500
6917/6917 [==============================] - 22s 3ms/step - loss: 0.3619 - accuracy: 0.9310
Epoch 310/500
6917/6917 [==============================] - 22s 3ms/step - loss: 0.3584 - accuracy: 0.9315
Epoch 311/500
6917/6917 [==============================] - 22s 3ms/step - loss: 0.3572 - accuracy: 0.9331
Epoch 312/500
6917/6917 [==============================] - 22s 3ms/step - loss: 0.3542 - accuracy: 0.9326
Epoch 313/500
6917/6917 [==============================] - 22s 3ms/step - loss: 0.3495 - accuracy: 0.9336
Epoch 314/500
6917/6917 [==============================] - 22s 3ms/step - loss: 0.3478 - accuracy: 0.9325
Epoch 315/500
6917/6917 [==============================] - 22s 3ms/step - loss: 0.3463 - accuracy: 0.9341
Epoch 316/500
6917/6917 [==============================] - 22s 3ms/step - loss: 0.3401 - accuracy: 0.9345
Epoch 317/500
6917/6917 [==============================] - 2

6917/6917 [==============================] - 22s 3ms/step - loss: 0.2056 - accuracy: 0.9523
Epoch 386/500
6917/6917 [==============================] - 22s 3ms/step - loss: 0.2042 - accuracy: 0.9523
Epoch 387/500
6917/6917 [==============================] - 23s 3ms/step - loss: 0.2037 - accuracy: 0.9520
Epoch 388/500
6917/6917 [==============================] - 22s 3ms/step - loss: 0.2024 - accuracy: 0.9514
Epoch 389/500
6917/6917 [==============================] - 22s 3ms/step - loss: 0.2007 - accuracy: 0.9526
Epoch 390/500
6917/6917 [==============================] - 22s 3ms/step - loss: 0.2001 - accuracy: 0.9526
Epoch 391/500
6917/6917 [==============================] - 22s 3ms/step - loss: 0.1983 - accuracy: 0.9536
Epoch 392/500
6917/6917 [==============================] - 22s 3ms/step - loss: 0.1983 - accuracy: 0.9521
Epoch 393/500
6917/6917 [==============================] - 22s 3ms/step - loss: 0.1953 - accuracy: 0.9524
Epoch 394/500
6917/6917 [==============================] - 2

6917/6917 [==============================] - 22s 3ms/step - loss: 0.1442 - accuracy: 0.9565
Epoch 463/500
6917/6917 [==============================] - 22s 3ms/step - loss: 0.1434 - accuracy: 0.9556
Epoch 464/500
6917/6917 [==============================] - 22s 3ms/step - loss: 0.1421 - accuracy: 0.9572
Epoch 465/500
6917/6917 [==============================] - 22s 3ms/step - loss: 0.1421 - accuracy: 0.9566
Epoch 466/500
6917/6917 [==============================] - 22s 3ms/step - loss: 0.1421 - accuracy: 0.9571
Epoch 467/500
6917/6917 [==============================] - 22s 3ms/step - loss: 0.1409 - accuracy: 0.9552
Epoch 468/500
6917/6917 [==============================] - 22s 3ms/step - loss: 0.1406 - accuracy: 0.9565
Epoch 469/500
6917/6917 [==============================] - 22s 3ms/step - loss: 0.1404 - accuracy: 0.9562
Epoch 470/500
6917/6917 [==============================] - 22s 3ms/step - loss: 0.1397 - accuracy: 0.9578
Epoch 471/500
6917/6917 [==============================] - 2

In [33]:
import matplotlib.pyplot as plt
plt.plot(hist.history['accuracy'])
plt.show()
model.save('tentGen.hdf5')

<Figure size 640x480 with 1 Axes>

In [39]:
from keras.models import load_model
model=load_model("C:/Users/student/Downloads/Python_JP/tentGen.hdf5")

In [45]:
wordList="대한민국 의 국민 이 되는 요건은 법률 로 정한 다".split()
wordList

['대한민국', '의', '국민', '이', '되는', '요건은', '법률', '로', '정한', '다']

In [47]:
reverseWordMap=dict(map(reversed,tokenizer.word_index.items()))
len(reverseWordMap) # 1164

1164

In [48]:
reverseWordMap

{1: '의',
 2: '에',
 3: '을',
 4: '은',
 5: '이',
 6: '한다',
 7: '·',
 8: '를',
 9: '법률',
 10: '는',
 11: '한',
 12: '할',
 13: '정',
 14: '수',
 15: '대통령',
 16: '과',
 17: '국가',
 18: '또는',
 19: '국회',
 20: '의하여',
 21: '하는',
 22: '국민',
 23: '하여',
 24: '로',
 25: '관',
 26: '있다',
 27: '때',
 28: '헌법',
 29: '그',
 30: '에는',
 31: '가',
 32: '으로',
 33: '와',
 34: '모든',
 35: '바',
 36: '위',
 37: '적',
 38: '하며',
 39: '하여야',
 40: '된',
 41: '아니',
 42: '기타',
 43: '가진다',
 44: '및',
 45: '장',
 46: '사항',
 47: '인',
 48: '에서',
 49: '권리',
 50: '안',
 51: '된다',
 52: '필요한',
 53: '없다',
 54: '의원',
 55: '자유',
 56: '정부',
 57: '선거',
 58: '임명',
 59: '하고',
 60: '되지',
 61: '직무',
 62: '국무총리',
 63: '경제',
 64: '의무',
 65: '조직',
 66: '자',
 67: '하',
 68: '국회의원',
 69: '임기',
 70: '의결',
 71: '공무원',
 72: '경우',
 73: '이상',
 74: '국무위원',
 75: '대법원',
 76: '하기',
 77: '범위',
 78: '이를',
 79: '다',
 80: '될',
 81: '받지',
 82: '법관',
 83: '보호',
 84: '정책',
 85: '정당',
 86: '헌법재판소',
 87: '재적',
 88: '회의',
 89: '항의',
 90: '관리',
 91: '진다',
 92: '노력',
 93: '보장',
 

In [49]:
x=sequence.pad_sequences([[tokenizer.word_index[w] for w in wordList[:2]]], maxlen=maxLen)
x

array([[  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0

In [52]:
p=model.predict(x)[0] # (1,1165)

In [53]:
#            0  1  2  3
x=np.array([2.5,1.5,0,1])
x.argsort() # [2,3,1,0] : 인덱스번호

array([2, 3, 1, 0], dtype=int64)

In [54]:
np.argsort(p)

array([ 23, 920, 891, ...,  22, 441, 440], dtype=int64)

In [55]:
idx=np.flip(np.argsort(p),0)
p[idx]

array([3.1570053e-01, 3.0375248e-01, 2.2626188e-01, ..., 9.7054810e-14,
       4.5143722e-14, 7.5880102e-16], dtype=float32)

In [56]:
for i in idx[:10]:
    print(reverseWordMap[i])

주권
영토
국민
모든
지출
종류
자
행위
경
내용


In [61]:
def predictWord(i,n):
    x=sequence.pad_sequences([[tokenizer.word_index[w] for w in wordList[:i]]],maxlen=maxLen)
    p=model.predict(x)[0]
    idx=np.flip(np.argsort(p),0)
    for j in idx[:n]:
        print(" ".join(wordList[:i]), reverseWordMap[j], "(p={:4.2f}%)".format(100*p[j]))

In [62]:
predictWord(1,3)# 대한민국' 다음 단어?, 3개
print("="*50)
predictWord(2,3)# 대한민국 의' 다음 단어?, 3개 추천
print("="*50)
predictWord(5,4)#'대한민국 의 국민 이 되는 

대한민국 의 (p=49.03%)
대한민국 은 (p=34.14%)
대한민국 헌법 (p=14.42%)
대한민국 의 주권 (p=31.57%)
대한민국 의 영토 (p=30.38%)
대한민국 의 국민 (p=22.63%)
대한민국 의 국민 이 되는 요건 (p=99.95%)
대한민국 의 국민 이 되는 되는 (p=0.04%)
대한민국 의 국민 이 되는 조직 (p=0.00%)
대한민국 의 국민 이 되는 근로자 (p=0.00%)
